## MONICA VILLASUSO - DELIVERY WEEK 6 - PARTE 1

### 1. From HTML

*Using only beautiful soap*

Save in a dataframe the next information using web scraping. Each row of the dataframe must have in different columns:

- The name of the title
- The id of the div where is the value scraped. If there is not id, then the value is must be numpy.nan
- The name of the tag where is the value scraped.
- The next scraped values in different rows: 
    - The value: "Este es el segundo párrafo"  --> Row 1
    - The url https://pagina1.xyz/ --> Row 2
    - The url https://pagina4.xyz/ --> Row 3
    - The url https://pagina5.xyz/ --> Row 4
    - The value "links footer-links" --> Row 5
    - The value "Este párrafo está en el footer" --> Row 6

In [632]:
html = """<html lang="es">
<head>
    <meta charset="UTF-8">
    <title>Página de prueba</title>
</head>
<body>
<div id="main" class="full-width">
    <h1>El título de la página</h1>
    <p>Este es el primer párrafo</p>
    <p>Este es el segundo párrafo</p>
    <div id="innerDiv">
        <div class="links">
            <a href="https://pagina1.xyz/">Enlace 1</a>
            <a href="https://pagina2.xyz/">Enlace 2</a>
        </div>
        <div class="right">
            <div class="links">
                <a href="https://pagina3.xyz/">Enlace 3</a>
                <a href="https://pagina4.xyz/">Enlace 4</a>
            </div>
        </div>
    </div>
    <div id="footer">
        <!-- El footer -->
        <p>Este párrafo está en el footer</p>
        <div class="links footer-links">
            <a href="https://pagina5.xyz/">Enlace 5</a>
        </div>
    </div>
</div>
</body>
</html>"""

In [633]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import bs4

In [634]:
#print(html)

In [635]:
# To show beautiful html
def show_html(html_str):
    print(BeautifulSoup(str(html_str), 'html.parser').prettify())

In [636]:
soup = BeautifulSoup(html, 'html.parser')
#show_html(soup.text)

In [637]:
# Construye un diccionario con los valores para luego, a partir de allí, construir el Dataframe final (con las columnas title, id,tag y value)
title_c = []
id_c=[]
tag_c=[]
value_c = []

def agregar(tit, iden, etiq, valor, tit_l, iden_l, tag_l, value_l):
    tit_l.append(tit)
    iden_l.append(iden)
    tag_l.append(etiq)
    value_l.append(valor)

In [638]:
# Busqueda del título
title_v = soup.head.find("title").get_text()

In [639]:
# Busqueda de elementos según enunciado
tags_l = ['a', 'p']
body_l = list(soup.body)

for i in range(len(body_l)):
    if (isinstance(body_l[i],bs4.element.Tag)) :
        bloque =body_l[i]
        id_v = bloque["id"]
        cla_v = bloque["class"]
        for n in bloque:
            if isinstance(n, bs4.element.Tag):
                if not n.attrs:
                    if n.name in tags_l:
                        tag_v= n.name
                        search_v = n.get_text()
                        agregar (title_v, id_v, tag_v, search_v,title_c,id_c,tag_c,value_c)
                else:  # el elemento del bloque tiene atributos
                    for tt in tags_l:
                        tag_v = tt
                        subbloque = list(n)
                        for elem in subbloque:
                            if isinstance(elem,bs4.element.Tag):
                                id_v = elem.attrs.get("id")
                                if elem.attrs:         
                                    if elem.find(tt) not in (-1, None):    # Si consigue el tag 
                                        if isinstance(elem,bs4.element.Tag):
                                            id_v = elem.attrs.get("id")
                                            cla_v = elem.attrs.get("class")
                                            agregar (title_v,id_v, tag_v,elem.find(tag_v).getText(),title_c,id_c,tag_c,value_c)
                                else:
                                    if tt == elem.name:
                                        cla_v = elem.attrs.get("class")
                                        id_v = elem.attrs.get("id")
                                        #id_v = elem.id
                                        agregar (title_v,id_v, elem.name,elem.getText(),title_c,id_c,tag_c,value_c)
                                   

#Buscar div
tags_div = 'div'
div_tag = soup.body.div
last_elem = div_tag.find_all(tags_div, class_="links footer-links")[0]
elem_id = last_elem.id
elem_tag = last_elem.name
elem_search = last_elem.attrs["class"]
agregar (title_v,elem_id, elem_tag,elem_search,title_c,id_c,tag_c,value_c)

In [640]:
# Construccion del Dataframe
result_dicc= {'title': title_c, 
        'id': id_c,
        'tag':tag_c,
        'value': value_c}
result_df = pd.DataFrame(result_dicc)
result_df.fillna(value=pd.np.nan, inplace=True)

In [641]:
# Dataframe Final
result_df

,title,id,tag,value
0,Página de prueba,main,p,Este es el primer párrafo
1,Página de prueba,main,p,Este es el segundo párrafo
2,Página de prueba,NaN,a,Enlace 1
3,Página de prueba,NaN,a,Enlace 3
4,Página de prueba,NaN,a,Enlace 5
5,Página de prueba,NaN,p,Este párrafo está en el footer
6,Página de prueba,NaN,div,"[links, footer-links]"
